# Install fission

With helm, grafana, opentelemetry, prometheus, and jaeger enabled.

Install cert-manager and open-telemetry operator.

In [1]:
# cert-manager
!kubectl apply -f https://github.com/jetstack/cert-manager/releases/latest/download/cert-manager.yaml

namespace/cert-manager created
customresourcedefinition.apiextensions.k8s.io/certificaterequests.cert-manager.io created
customresourcedefinition.apiextensions.k8s.io/certificates.cert-manager.io created
customresourcedefinition.apiextensions.k8s.io/challenges.acme.cert-manager.io created
customresourcedefinition.apiextensions.k8s.io/clusterissuers.cert-manager.io created
customresourcedefinition.apiextensions.k8s.io/issuers.cert-manager.io created
customresourcedefinition.apiextensions.k8s.io/orders.acme.cert-manager.io created
serviceaccount/cert-manager-cainjector created
serviceaccount/cert-manager created
serviceaccount/cert-manager-webhook created
clusterrole.rbac.authorization.k8s.io/cert-manager-cainjector created
clusterrole.rbac.authorization.k8s.io/cert-manager-controller-issuers created
clusterrole.rbac.authorization.k8s.io/cert-manager-controller-clusterissuers created
clusterrole.rbac.authorization.k8s.io/cert-manager-controller-certificates created
clusterrole.rbac.autho

In [2]:
# open telemetry operator
!kubectl apply -f https://github.com/open-telemetry/opentelemetry-operator/releases/latest/download/opentelemetry-operator.yaml

namespace/opentelemetry-operator-system created
customresourcedefinition.apiextensions.k8s.io/instrumentations.opentelemetry.io created
customresourcedefinition.apiextensions.k8s.io/opampbridges.opentelemetry.io created
customresourcedefinition.apiextensions.k8s.io/opentelemetrycollectors.opentelemetry.io created
serviceaccount/opentelemetry-operator-controller-manager created
role.rbac.authorization.k8s.io/opentelemetry-operator-leader-election-role created
clusterrole.rbac.authorization.k8s.io/opentelemetry-operator-manager-role created
clusterrole.rbac.authorization.k8s.io/opentelemetry-operator-metrics-reader created
clusterrole.rbac.authorization.k8s.io/opentelemetry-operator-proxy-role created
rolebinding.rbac.authorization.k8s.io/opentelemetry-operator-leader-election-rolebinding created
clusterrolebinding.rbac.authorization.k8s.io/opentelemetry-operator-manager-rolebinding created
clusterrolebinding.rbac.authorization.k8s.io/opentelemetry-operator-proxy-rolebinding created
serv

Create opentelemetry operator:

In [3]:
%%bash
kubectl apply -f ./opentelemetry.yaml


configmap/otel-collector-conf created
service/otel-collector created
deployment.apps/otel-collector created


Create jaeger namespace:

In [4]:
%%bash
kubectl create namespace observability
kubectl create -n observability -f https://github.com/jaegertracing/jaeger-operator/releases/download/v1.39.0/jaeger-operator.yaml

namespace/observability created
customresourcedefinition.apiextensions.k8s.io/jaegers.jaegertracing.io created
serviceaccount/jaeger-operator created
role.rbac.authorization.k8s.io/leader-election-role created
role.rbac.authorization.k8s.io/prometheus created
clusterrole.rbac.authorization.k8s.io/jaeger-operator-metrics-reader created
clusterrole.rbac.authorization.k8s.io/manager-role created
clusterrole.rbac.authorization.k8s.io/proxy-role created
rolebinding.rbac.authorization.k8s.io/leader-election-rolebinding created
rolebinding.rbac.authorization.k8s.io/prometheus created
clusterrolebinding.rbac.authorization.k8s.io/jaeger-operator-proxy-rolebinding created
clusterrolebinding.rbac.authorization.k8s.io/manager-rolebinding created
service/jaeger-operator-metrics created
service/jaeger-operator-webhook-service created
deployment.apps/jaeger-operator created
certificate.cert-manager.io/jaeger-operator-serving-cert created
issuer.cert-manager.io/jaeger-operator-selfsigned-issuer create

Create jaeger instance:

In [5]:
%%bash
kubectl apply -n observability -f - <<EOF
apiVersion: jaegertracing.io/v1
kind: Jaeger
metadata:
  name: jaeger
EOF

jaeger.jaegertracing.io/jaeger created


Port forward jaeger service(run in external terminal):

In [ ]:
%%bash
kubectl port-forward service/jaeger-query -n observability 31931:16686 &

Install fission

In [6]:
%%bash
export FISSION_NAMESPACE=fission
kubectl create -k "github.com/fission/fission/crds/v1?ref=v1.20.1"
kubectl create namespace $FISSION_NAMESPACE
helm install --namespace $FISSION_NAMESPACE \
  fission fission-charts/fission-all \
  --set openTelemetry.otlpCollectorEndpoint="otel-collector.opentelemetry-operator-system.svc:4317" \
  --set openTelemetry.otlpInsecure=true \
  --set openTelemetry.tracesSampler="parentbased_traceidratio" \
  --set openTelemetry.tracesSamplingRate="1"

customresourcedefinition.apiextensions.k8s.io/canaryconfigs.fission.io created
customresourcedefinition.apiextensions.k8s.io/environments.fission.io created
customresourcedefinition.apiextensions.k8s.io/functions.fission.io created
customresourcedefinition.apiextensions.k8s.io/httptriggers.fission.io created
customresourcedefinition.apiextensions.k8s.io/kuberneteswatchtriggers.fission.io created
customresourcedefinition.apiextensions.k8s.io/messagequeuetriggers.fission.io created
customresourcedefinition.apiextensions.k8s.io/packages.fission.io created
customresourcedefinition.apiextensions.k8s.io/timetriggers.fission.io created
namespace/fission created


W0620 11:31:11.536318   28688 warnings.go:70] metadata.name: this is used in Pod names and hostnames, which can result in surprising behavior; a DNS label is recommended: [must not contain dots]


NAME: fission
LAST DEPLOYED: Thu Jun 20 11:31:10 2024
NAMESPACE: fission
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
1. Install the client CLI.

Mac:
  $ curl -Lo fission https://github.com/fission/fission/releases/download/v1.20.2/fission-v1.20.2-darwin-amd64 && chmod +x fission && sudo mv fission /usr/local/bin/

Linux:
  $ curl -Lo fission https://github.com/fission/fission/releases/download/v1.20.2/fission-v1.20.2-linux-amd64 && chmod +x fission && sudo mv fission /usr/local/bin/

Windows:
  For Windows, you can use the linux binary on WSL. Or you can download this windows executable: https://github.com/fission/fission/releases/download/v1.20.2/fission-v1.20.2-windows-amd64.exe

2. You're ready to use Fission!
  You can create fission resources in the namespace "default"

  # Create an environment
  $ fission env create --name nodejs --image fission/node-env --namespace default

  # Get a hello world
  $ curl https://raw.githubusercontent.com/fission/examples/master/nodejs

Create grafana namespace:

In [7]:
%%bash
export METRICS_NAMESPACE=monitoring
kubectl create namespace $METRICS_NAMESPACE

namespace/monitoring created


Install grafana:

In [8]:
%%bash
helm repo add prometheus-community https://prometheus-community.github.io/helm-charts
helm repo update
helm install prometheus prometheus-community/kube-prometheus-stack -n monitoring

"prometheus-community" already exists with the same configuration, skipping
Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "prometheus-community" chart repository
...Successfully got an update from the "fission-charts" chart repository
...Successfully got an update from the "budibase" chart repository
...Unable to get an update from the "bitnami" chart repository (https://charts.bitnami.com/bitnami):
	Get "https://charts.bitnami.com/bitnami/index.yaml": EOF
Update Complete. ⎈Happy Helming!⎈
NAME: prometheus
LAST DEPLOYED: Thu Jun 20 11:31:33 2024
NAMESPACE: monitoring
STATUS: deployed
REVISION: 1
NOTES:
kube-prometheus-stack has been installed. Check its status by running:
  kubectl --namespace monitoring get pods -l "release=prometheus"

Visit https://github.com/prometheus-operator/kube-prometheus for instructions on how to create & configure Alertmanager and Prometheus instances using the Operator.


Upgrade fission using helm:

In [9]:
!helm upgrade fission fission-charts/fission-all --namespace fission -f values.yaml

W0620 11:33:10.410498   28708 warnings.go:70] metadata.name: this is used in Pod names and hostnames, which can result in surprising behavior; a DNS label is recommended: [must not contain dots]
W0620 11:33:14.188388   28708 warnings.go:70] metadata.name: this is used in Pod names and hostnames, which can result in surprising behavior; a DNS label is recommended: [must not contain dots]
Release "fission" has been upgraded. Happy Helming!
NAME: fission
LAST DEPLOYED: Thu Jun 20 11:33:08 2024
NAMESPACE: fission
STATUS: deployed
REVISION: 2
TEST SUITE: None
NOTES:
1. Install the client CLI.

Mac:
  $ curl -Lo fission https://github.com/fission/fission/releases/download/v1.20.2/fission-v1.20.2-darwin-amd64 && chmod +x fission && sudo mv fission /usr/local/bin/

Linux:
  $ curl -Lo fission https://github.com/fission/fission/releases/download/v1.20.2/fission-v1.20.2-linux-amd64 && chmod +x fission && sudo mv fission /usr/local/bin/

Windows:
  For Windows, you can use the linux binary on WSL

Expose grafana service(run in external terminal):

In [ ]:
%%bash
kubectl --namespace monitoring port-forward svc/prometheus-grafana 39131:80 &

Get grafana password:

In [10]:
%%bash
kubectl get secret --namespace monitoring prometheus-grafana -o jsonpath="{.data.admin-password}" | base64 --decode ; echo

prom-operator
